<!-- Autoheader begin -->
<hr/>
<div id="navtitle_3_1_jl" style="text-align:center; font-size:16px">III.1 Population Inversion in a Two-Level-System using Krotov's Method and GRAPE</div>
<hr/>
<table style="width: 100%">
  <tr>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_2_3_chiral.ipynb">$\leftarrow$ previous notebook </a><br>
        <a href="jl_exercise_2_3_chiral.ipynb" style="font-size:13px">II.3 Parameter Optimization of Three-Wave Mixing in a Three-Level System</a>
    </th>
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_2_1_TLS.ipynb">$\uparrow$ previous part $\uparrow$</a><br>
        <a href="jl_exercise_2_1_TLS.ipynb" style="font-size:13px">II.1 Population Inversion in a Two-Level-System using Parameter Optimization</a>
    </td>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_3_2_lambda.ipynb">next notebook $\rightarrow$</a><br>
        <a href="jl_exercise_3_2_lambda.ipynb" style="font-size:13px">III.2 Optimal Control for STIRAP</a>
    </th>
  </tr>
  <tr style="width: 100%">
    <td style="width:33%; text-align:center; font-size:16px">
    </td>
  </tr>
</table>

<div style="text-align: right;font-size: 16px"><a href="../Python/py_exercise_3_1_TLS.ipynb">👉 Python version</a></div>

---
<!-- Autoheader end -->

# Population Inversion in a Two-Level-System using Krotov's Method and GRAPE

$\newcommand{tr}[0]{\operatorname{tr}}
\newcommand{diag}[0]{\operatorname{diag}}
\newcommand{abs}[0]{\operatorname{abs}}
\newcommand{pop}[0]{\operatorname{pop}}
\newcommand{aux}[0]{\text{aux}}
\newcommand{opt}[0]{\text{opt}}
\newcommand{tgt}[0]{\text{tgt}}
\newcommand{init}[0]{\text{init}}
\newcommand{lab}[0]{\text{lab}}
\newcommand{rwa}[0]{\text{rwa}}
\newcommand{bra}[1]{\langle#1\vert}
\newcommand{ket}[1]{\vert#1\rangle}
\newcommand{Bra}[1]{\left\langle#1\right\vert}
\newcommand{Ket}[1]{\left\vert#1\right\rangle}
\newcommand{Braket}[2]{\left\langle #1\vphantom{#2} \mid
#2\vphantom{#1}\right\rangle}
\newcommand{op}[1]{\hat{#1}}
\newcommand{Op}[1]{\hat{#1}}
\newcommand{dd}[0]{\,\text{d}}
\newcommand{Liouville}[0]{\mathcal{L}}
\newcommand{DynMap}[0]{\mathcal{E}}
\newcommand{identity}[0]{\mathbf{1}}
\newcommand{Norm}[1]{\lVert#1\rVert}
\newcommand{Abs}[1]{\left\vert#1\right\vert}
\newcommand{avg}[1]{\langle#1\rangle}
\newcommand{Avg}[1]{\left\langle#1\right\rangle}
\newcommand{AbsSq}[1]{\left\vert#1\right\vert^2}
\newcommand{Re}[0]{\operatorname{Re}}
\newcommand{Im}[0]{\operatorname{Im}}$

This notebook is the first illustration of using a complete optimal control framework with
gradient-based methods. It considers the example of inverting the
population in a two-level system that we have already explored in 
[Exercise I.1](jl_exercise_1_1_TLS.ipynb) and 
[Exercise II.1](jl_exercise_2_1_TLS.ipynb). These previous notebooks were aimed at finding control solutions by
tuning a small number of analytical parameters for a simple pulse shape,
either by hand, or by using a gradient-free numerical optimization method, respectively.

In this notebook, you will learn how to use numerical optimization to find
control fields $E(t)$ which are not limited to a simple, predefined analytical shape.
Instead, we treat the value of $E(t)$ at every point $t$ of a finely sampled time grid as an independent
control parameter. This allows for a lot of control parameters (as many as
there are points on the time grid) and thus a lot of freedom to find optimal solutions. Gradient-free methods are only suitable for a small amount of control parameters and are thus not appropriate for such a task. Instead, we employ methods which take into account the *gradient* of the optimization functional with respect to the control parameters (the values $E(t)$). Two of the more established methods for gradient-based optimization of fields $E(t)$ (approximated as piecewise-constant on a fine time grid) are "GRadient Ascent Pulse Engineering" (GRAPE) and
Krotov's method.

You will learn here the use of the [`QuantumControl` Julia
framework](https://juliaquantumcontrol.github.io/QuantumControl.jl) to
formulate and solve the control problem.

In [ ]:
using QuantumControl

Specifically, we'll be using Krotov's method as implemented in the `Krotov`
(sub-)package.

In [ ]:
using Krotov

Or, optionally (as a "bonus"), you can also learn how to use GRAPE.

In [ ]:
using GRAPE

Both methods work equally well in this example example. We use Krotov's method as the default to match the workflow in the corresponding Python notebook.


## Setup

For simulating the dynamics, we will use the `propagate` function from the `QuantumPropagators` package. In the context of optimal control with Krotov's method, control fields must be treated as piecewise constant. For a small system like in this example, the simplest and most efficient approach for the time propagation is to construct a time evolution operator in each time step using explicit matrix exponentiation. This is implemented as `ExpProp` in `QuantumPropagators`:

In [ ]:
using QuantumPropagators: propagate
using QuantumPropagators: ExpProp

For visualization, we will use the `Plots` package

In [ ]:
using Plots

# Set up thicker default lines in plots
Plots.default(
    linewidth               = 2.0,
    foreground_color_legend = nothing,
    background_color_legend = RGBA(1, 1, 1, 0.8)
)

## Define the Hamiltonian

In the
following the Hamiltonian, guess field, and states are defined.

The Hamiltonian
$\op{H}_{0} = - \frac{\omega}{2} \op{\sigma}_{z}$
represents a
simple qubit with energy
level splitting $\omega$ in the canonical basis
$\{\ket{0},\ket{1}\}$. The control
field
$\epsilon(t)$ is assumed to couple via
the
Hamiltonian $\op{H}_{1}(t) =
\epsilon(t) \op{\sigma}_{x}$ to the qubit,
i.e., the control
field effectively
drives
transitions between the two basis states.

In [ ]:
"""Two-level-system Hamiltonian

# Keyword Arguments

* `omega` (float): energy separation of the qubit levels
* `eps0` (function): amplitude eps0(t) of the driving field
"""
function ham_and_states(; omega=1.0, eps0=(t -> 1.0))

    H₀ = -0.5 * omega * [
        1   0
        0  -1
    ]
    H₁ = Float64[
        0  1
        1  0
    ]

    Ψ₀ = ComplexF64[1, 0] # State |0⟩
    Ψ₁ = ComplexF64[0, 1] # State |1⟩

    H = hamiltonian(H₀, (H₁, eps0))

    return H, Ψ₀, Ψ₁

end

In addition, we define a shape function $S(t)$ which takes care of
experimental limits such as the necessity of finite ramps
at the beginning and
end of the control field.

In [ ]:
"""Shape function for the field update"""
S(t) = QuantumControl.Shapes.flattop(t; T=10.0, t_rise=0.5, func=:sinsq);

This shape function will be used in two contexts: First, in the optimization with Krotov's method later on in this tutorial, it will shape the pulse update, ensuring that the boundary conditions are maintained in every iteration of the optimization.

Second, we will also use $S(t)$ to multiply the guess field `eps0` when calling `ham_and_states` while simulating the dynamics under the guess pulse.

## Simulate dynamics of the guess field

Before heading towards the optimization
procedure, we first simulate the
dynamics under the guess field
$\epsilon_{0}(t)$.

However, before we can propagate the state under the guess field, we need to define the time grid of the
dynamics. As an example, we define the
initial state to be at time $t=0$ and
consider a total propagation time of
$T=10$. The entire time grid is divided into
$n_{t}=80$ equidistant time steps (corresponding to 81 time grid points).

In [ ]:
tlist = collect(range(0, 10; length=81));

Naturally, we also have to define the guess pulse itself.

In [ ]:
"""Guess Amplitude (unshaped)"""
E(t; A=0.1, σ=2) = A * exp(-(t-5)^2 / (2 * σ^2)) * cos(3t);

In the Hamiltonian, we multiply the guess `E(t)` with the shape `S(t)` as mentioned above:

In [ ]:
H, Ψ₀, Ψ₁ = ham_and_states(eps0=(t -> S(t) * E(t)));

Feel free to play around with the definition of `E(t)` and the construction of the Hamiltonian.

The total field looks as follows:

In [ ]:
using QuantumControl.Controls: discretize

function plot_pulse(pulse, tlist)
    fig = plot(; xlabel="time", ylabel="pulse amplitude")
    plot!(fig, tlist, discretize(pulse, tlist); label="")
    return fig
end

plot_pulse(H.amplitudes[1], tlist)

Then, we solve the equation of motion for the initial state $\ket{\Psi_{\init}}=\ket{0}$ and the Hamiltonian $\op{H}(t)$
generating its evolution. Afterwards, we plot the population dynamics.

In [ ]:
states = propagate(Ψ₀, H, tlist; method=ExpProp, storage=true);

The `storage` parameter tells `propagate` to return a concatenated array of shape `2×nt` containing the two-level state at each point in time. We can plot the population in both levels in one go, by applying the absolute-square function elementwise to the states:

In [ ]:
plot(abs2.(states)', labels=["0" "1"]; xlabel="time", ylabel="population")

Note that there is a small amount of oscillation in the dynamics, which we can see by zooming in the dynamics of the $\ket{1}$ state

In [ ]:
plot(abs2.(states[2,:]), label="1"; xlabel="time", ylabel="population")

We can do the same again, but plot the trajectory on the Bloch sphere instead. For that, we will need the expectation values of the Pauli operators, instead of the states or the population in these levels. Thus, we construct a list of `observables` and repeat the propagation:

In [ ]:
const 𝕚 = 1im
σ_x = ComplexF64[0 1; 1 0]
σ_y = ComplexF64[0 -𝕚; 𝕚 0]
σ_z = ComplexF64[1 0; 0 -1]

bloch_vals = propagate(Ψ₀, H, tlist; method=ExpProp, observables=[σ_x, σ_y, σ_z], storage=true);

The Python [QuTiP](https://qutip.org) package contains some nice [Bloch sphere visualizations](https://qutip.org/docs/4.1/guide/guide-bloch.html), which unfortunately no one has ported to Julia yet. Luckily, Julia can call Python code with the help of the `PythonCall` package. We'll set that up here under the assumption that we can use the Python project environment for this tutorial. Please make sure that the `JULIA_PYTHONCALL_EXE` environment variable points to a `python` executable in an environment that has `qutip` intalled (*before* you execute `using PythonCall`, otherwise you'll have to restart the kernel for this notebook and rerun everything):

In [ ]:
ENV["JULIA_CONDAPKG_BACKEND"] = "Null"
ENV["JULIA_PYTHONCALL_EXE"] = joinpath("..", ".venv",  Sys.iswindows() ? "python.exe" : "bin/python")

using PythonCall: Py, pyimport

Ideally, we should now be able to import the `qutip` and `matplotlib` Python modules:

In [ ]:
qutip = pyimport("qutip");
matplotlib = pyimport("matplotlib");

If you have problems getting this to run, this part is not too important for the rest of the notebook and you may skip this section for the time being.

In [ ]:
function plot_bloch(bloch_vals; view=[-60, 30])
    b = qutip.Bloch(view=view)
    exp_x = Py(bloch_vals[1,:]).to_numpy()
    exp_y = Py(bloch_vals[2,:]).to_numpy()
    exp_z = Py(bloch_vals[3,:]).to_numpy()
    b.add_points([exp_x, exp_y, exp_z], "m")
    b.point_color = matplotlib.pyplot.get_cmap("viridis_r")(
        Py(range(0, 1; length=length(exp_x))).to_numpy()
    )
    b.frame_alpha = 0.1
    b.make_sphere()
    return b
end

plot_bloch(bloch_vals)

## Define the optimization target

We want to optimize a simple state-to-state
transfer
from the initial state $\ket{\Psi_{\init}} = \ket{0}$ to the target state
$\ket{\Psi_{\tgt}} = \ket{1}$, which we want to reach at final time $T$. Note
that we also have to pass the Hamiltonian $\op{H}(t)$ which generates the
dynamics of
the system.

From a mathematical perspective we optimize the guess field $\epsilon_{0}(t)$ such
that the intended state-to-state transfer $\ket{\Psi_{\init}} \rightarrow
\ket{\Psi_{\tgt}}$ is obtained.
To this end, we
choose the functional to be $F = F_{ss}$ with
\begin{equation}
F_{ss}
=
\left|\Braket{\Psi(T)}{\Psi_{\tgt}}\right|^2
\end{equation}

with
$\ket{\Psi(T)}$ the
forward propagated state of $\ket{\Psi_{\init}}$. Maximizing $F_{ss}$ is equivalent to minimizing the infidelity which we employ as our optimization functional, i.e.

In [ ]:
using QuantumControl.Functionals: J_T_ss

print(@doc J_T_ss)  # You could also do `? J_T_ss` to access the help system

The functional is evaluated based on the initial state forward-propagated under a specific time-dependent Hamiltonian $\Op{H}(t)$ and considers the overlap with the associated target state. The initial state, Hamiltonian, and target state are collected in an `Trajectory` object that will be passed to the `J_T_ss` function.

In [ ]:
trajectories = [Trajectory(Ψ₀, H; target_state=Ψ₁)];

The *result* of the propagation is available to `J_T_ss` as the first positional argument (`ϕ`).

To fully define the optimization problem, we put everything together as

In [ ]:
problem = ControlProblem(
    trajectories,
    tlist;
    prop_method=ExpProp,
    J_T=J_T_ss,
    iter_stop=50,
);

## Using Krotov's method

In an optimization with Krotov's method, we have the option of using $S(t)$ as the `update_shape` for
$\epsilon_0(t)$. Wherever $S(t)$ is zero, the optimization will not change the
value of the control from the original guess. In general, this shape function can be different from the one used to shape the guess pulse. In addition, we have to choose `lambda_a` for each control
field. This parameter controls the magnitude of the updates for the respective fields in each iterative step of the optimization algorithm.

The `update_shape` and `lambda_a` arguments could have been given in the construction of the `ControlProblem` above. Here, we give them only now when moving to solve the `problem` using Krotov's method:

In [ ]:
res = optimize(problem; method=Krotov, lambda_a=25, update_shape=S)

## Simulate dynamics of the optimized field

Having obtained the optimized
control field, we can now
plot it and calculate the
population dynamics under
this field.

In [ ]:
using QuantumPropagators.Controls: get_controls, substitute

In [ ]:
H_opt = substitute(H, Dict(get_controls(H)[1] => res.optimized_controls[1]));

In [ ]:
states_opt = propagate(Ψ₀, H_opt, tlist; method=ExpProp, storage=true);

In [ ]:
plot_pulse(H_opt.amplitudes[1], tlist)

In [ ]:
plot(abs2.(states_opt)', labels=["0" "1"]; xlabel="time", ylabel="population", legend=:right)

In [ ]:
bloch_vals = propagate(Ψ₀, H_opt, tlist; method=ExpProp, observables=[σ_x, σ_y, σ_z], storage=true);

plot_bloch(bloch_vals)

## Further Tasks

1) Vary the numerical parameters `lambda_a` and $n_{t}$ to improve the optimization. You should be able to reach the desired fidelity of 99% within less than 50 iterations.

2) Try to improve the guess pulse to speed up the convergence. Hint: The interesting parameters are `A` and $T$ (Keep in mind to change it in the shape $S$ and in the time grid `tlist`). Note that a constant pulse might not be
the best option as a guess pulse.

## Bonus: Using GRAPE

Instead of Krotov's method, we can also use another popular gradient-based method in the toolbox of optimal control, GRadient Ascent Pulse Engineering (GRAPE):

The GRAPE method does not allow for an `update_shape` like in Krotov's method in order to guarantee that the boundary conditions for the control field are maintained in each iteration. However, we can work around that by separating the "control amplitude" in the Hamiltonian into a static shape $S(t)$ that the optimization will not alter, multiplied with the control $E(t)$ that can be modified in an arbitrary way. The `QuantumControl` package provides a `ShapedAmplitude` for such a case:

In [ ]:
using QuantumControl.Amplitudes: ShapedAmplitude
H, Ψ₀, Ψ₁ = ham_and_states(eps0=ShapedAmplitude(E, tlist; shape=S));

In [ ]:
problem = ControlProblem(
    [Trajectory(Ψ₀, H; target_state=Ψ₁)],
    tlist;
    prop_method=ExpProp,
    J_T=J_T_ss,
    iter_stop=50,
);

GRAPE also allows us to put lower and upper bounds on the controls. For very "simple" control problems such as this one, the bounds also help GRAPE not to overshoot (since the L-BFGS-B optimization method used internally in GRAPE by default has no settings to influence the linesearch that determines how far to go in the direction of the gradient). Feel free to experiment with the values for the bounds.

In [ ]:
res_grape = optimize(problem; method=GRAPE, lower_bound=-0.5, upper_bound=0.5)

Note that we could have employed the `ShapedAmplitude` for Krotov's method as well. This would have allowed us to use the (default) `update_shape=(t -> 1.0)` which places no restrictions on the update. If you like, you can return to the optimization with Krotov and check that doing this indeed gives a similar result as the original optimization above. If you would like to check your understanding of the intricacies of Krotov's method, ask yourself why it does not give *exactly* the same result.

The result of the GRAPE optimization (using appropriate bounds) is similar to the result obtained with Krotov's method:

In [ ]:
using QuantumControl.Controls: discretize_on_midpoints

H_opt = substitute(H, Dict(get_controls(H)[1] => discretize_on_midpoints(res_grape.optimized_controls[1], tlist)));

plot_pulse(pulse::ShapedAmplitude, tlist) = plot_pulse(Array(pulse), tlist)

plot_pulse(H_opt.amplitudes[1], tlist)

In [ ]:
states_opt = propagate(Ψ₀, H_opt, tlist; method=ExpProp, storage=true);

In [ ]:
plot(abs2.(states_opt)', labels=["0" "1"]; xlabel="time", ylabel="population", legend=:right)

In [ ]:
bloch_vals = propagate(Ψ₀, H_opt, tlist; method=ExpProp, observables=[σ_x, σ_y, σ_z], storage=true);

plot_bloch(bloch_vals)

## Next steps

You are now ready to advance to a more sophisticated example for gradient-based optimization. For instance, [Exercise III.2](jl_exercise_3_2_lambda.ipynb) covers the STIRAP protocol in the three-level lambda system. If you are more interested in gate optimization for quantum information applications, we recommend [Exercise III.4](jl_exercise_3_4_gate.ipynb).

<!-- Autofooter begin -->

---

[⬆︎ jump to top](#navtitle_3_1_jl)
<!-- Autofooter end -->